In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import plotly.express as px
import polars as pl
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x :.2f}')
from cleaner import DataCleaner
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from tools import (
    logging,
    bins_generator,
    import_datasets,
    decode_clean_actors,
    decode_clean,
    person_dataframe,

)
import eda_movies as eda

In [10]:
# link = "big_dataframe/actors.parquet"
link = "movies_datasets/name_basics.tsv"
test = import_datasets(link, 'pandas', sep="\t")
test

2023-10-27 09:35:24 INFO     Pandas loaded ! Importing name_basics...


nconst         primaryName birthYear deathYear  \
0         nm0000001        Fred Astaire      1899      1987   
1         nm0000002       Lauren Bacall      1924      2014   
2         nm0000003     Brigitte Bardot      1934        \N   
3         nm0000004        John Belushi      1949      1982   
4         nm0000005      Ingmar Bergman      1918      2007   
...             ...                 ...       ...       ...   
12933862  nm9993714   Romeo del Rosario        \N        \N   
12933863  nm9993716       Essias Loberg        \N        \N   
12933864  nm9993717  Harikrishnan Rajan        \N        \N   
12933865  nm9993718         Aayush Nair        \N        \N   
12933866  nm9993719          Andre Hill        \N        \N   

                            primaryProfession  \
0              soundtrack,actor,miscellaneous   
1                          actress,soundtrack   
2         actress,soundtrack,music_department   
3                     actor,soundtrack,writer   
4                       writer,director,actor   
...                                       ...   
12933862  animation_department,art_department   
12933863                                  NaN   
12933864                      cinematographer   
12933865                      cinematographer   
12933866                                  NaN   

                                   knownForTitles  
0         tt0050419,tt0072308,tt0053137,tt0031983  
1         tt0075213,tt0037382,tt0038355,tt0117057  
2         tt0054452,tt0057345,tt0049189,tt0056404  
3         tt0080455,tt0072562,tt0078723,tt0077975  
4         tt0083922,tt0050976,tt0050986,tt0069467  
...                                           ...  
12933862          tt2455546,tt11657662,tt14069590  
12933863                                       \N  
12933864                                tt8736744  
12933865                                tt8736744  
12933866                                       \N  

[12933867 rows x 6 columns]

In [11]:
df = test.copy()

In [12]:
df

nconst         primaryName birthYear deathYear  \
0         nm0000001        Fred Astaire      1899      1987   
1         nm0000002       Lauren Bacall      1924      2014   
2         nm0000003     Brigitte Bardot      1934        \N   
3         nm0000004        John Belushi      1949      1982   
4         nm0000005      Ingmar Bergman      1918      2007   
...             ...                 ...       ...       ...   
12933862  nm9993714   Romeo del Rosario        \N        \N   
12933863  nm9993716       Essias Loberg        \N        \N   
12933864  nm9993717  Harikrishnan Rajan        \N        \N   
12933865  nm9993718         Aayush Nair        \N        \N   
12933866  nm9993719          Andre Hill        \N        \N   

                            primaryProfession  \
0              soundtrack,actor,miscellaneous   
1                          actress,soundtrack   
2         actress,soundtrack,music_department   
3                     actor,soundtrack,writer   
4                       writer,director,actor   
...                                       ...   
12933862  animation_department,art_department   
12933863                                  NaN   
12933864                      cinematographer   
12933865                      cinematographer   
12933866                                  NaN   

                                   knownForTitles  
0         tt0050419,tt0072308,tt0053137,tt0031983  
1         tt0075213,tt0037382,tt0038355,tt0117057  
2         tt0054452,tt0057345,tt0049189,tt0056404  
3         tt0080455,tt0072562,tt0078723,tt0077975  
4         tt0083922,tt0050976,tt0050986,tt0069467  
...                                           ...  
12933862          tt2455546,tt11657662,tt14069590  
12933863                                       \N  
12933864                                tt8736744  
12933865                                tt8736744  
12933866                                       \N  

[12933867 rows x 6 columns]

In [13]:
df.drop(["deathYear", "primaryProfession"], axis=1, inplace=True)

In [14]:
clean = DataCleaner()
clean.fix_values(df, "fix_n")

2023-10-27 09:35:44 INFO     Fixing N values...


nconst            None
primaryName       None
birthYear         None
knownForTitles    None
dtype: object

In [15]:
df

nconst         primaryName birthYear  \
0         nm0000001        Fred Astaire      1899   
1         nm0000002       Lauren Bacall      1924   
2         nm0000003     Brigitte Bardot      1934   
3         nm0000004        John Belushi      1949   
4         nm0000005      Ingmar Bergman      1918   
...             ...                 ...       ...   
12933862  nm9993714   Romeo del Rosario         0   
12933863  nm9993716       Essias Loberg         0   
12933864  nm9993717  Harikrishnan Rajan         0   
12933865  nm9993718         Aayush Nair         0   
12933866  nm9993719          Andre Hill         0   

                                   knownForTitles  
0         tt0050419,tt0072308,tt0053137,tt0031983  
1         tt0075213,tt0037382,tt0038355,tt0117057  
2         tt0054452,tt0057345,tt0049189,tt0056404  
3         tt0080455,tt0072562,tt0078723,tt0077975  
4         tt0083922,tt0050976,tt0050986,tt0069467  
...                                           ...  
12933862          tt2455546,tt11657662,tt14069590  
12933863                                        0  
12933864                                tt8736744  
12933865                                tt8736744  
12933866                                        0  

[12933867 rows x 4 columns]

In [16]:
df["knownForTitles"] = np.where(df["knownForTitles"] == 0, "Unknown", df["knownForTitles"])
df["knownForTitles"] = df["knownForTitles"].str.split(",")
# mf["person_role"] = mf["person_role"].apply(decode_clean_actors).str.split(",")


In [17]:
df["birthYear"] = df["birthYear"].astype("int64")

In [18]:
df.head()

nconst      primaryName  birthYear  \
0  nm0000001     Fred Astaire       1899   
1  nm0000002    Lauren Bacall       1924   
2  nm0000003  Brigitte Bardot       1934   
3  nm0000004     John Belushi       1949   
4  nm0000005   Ingmar Bergman       1918   

                                 knownForTitles  
0  [tt0050419, tt0072308, tt0053137, tt0031983]  
1  [tt0075213, tt0037382, tt0038355, tt0117057]  
2  [tt0054452, tt0057345, tt0049189, tt0056404]  
3  [tt0080455, tt0072562, tt0078723, tt0077975]  
4  [tt0083922, tt0050976, tt0050986, tt0069467]

In [26]:
df.to_parquet("clean_datasets/person.parquet")

In [10]:
# link = "clean_datasets/movies_cleaned.parquet"
link_principals = "movies_datasets/title_principals.tsv"

# movies_cleaned = import_datasets(link, "parquet")
principals = import_datasets(link_principals, "pandas", sep="\t")

2023-10-26 20:49:03 INFO     Pandas loaded ! Importing title_principals...


In [11]:
df_prin = principals.copy()

In [12]:
df_prin.drop(["job"], inplace=True, axis=1)


In [13]:
df_prin

tconst  ordering     nconst         category characters
0         tt0000001         1  nm1588970             self   ["Self"]
1         tt0000001         2  nm0005690         director         \N
2         tt0000001         3  nm0374658  cinematographer         \N
3         tt0000002         1  nm0721526         director         \N
4         tt0000002         2  nm1335271         composer         \N
...             ...       ...        ...              ...        ...
58672901  tt9916880         5  nm0584014         director         \N
58672902  tt9916880         6  nm0996406         director         \N
58672903  tt9916880         7  nm1482639           writer         \N
58672904  tt9916880         8  nm2586970           writer         \N
58672905  tt9916880         9  nm1594058         producer         \N

[58672906 rows x 5 columns]

In [14]:
df_prin = clean.fix_values(df_prin, "fix_n")


2023-10-26 20:49:45 INFO     Fixing N values...


In [15]:

df_prin["characters"] = np.where(df_prin["characters"] == 0, "Unknown", df_prin["characters"])
# df_prin["characters"] = df_prin["characters"].str.split(",")
df_prin["characters"] = df_prin["characters"].apply(decode_clean_actors).str.split(",")


In [16]:
df_prin["category"].unique()

array(['self', 'director', 'cinematographer', 'composer', 'producer',
       'editor', 'actor', 'actress', 'writer', 'production_designer',
       'archive_footage', 'archive_sound'], dtype=object)

In [17]:

test = df_prin.copy()

In [19]:
actors_list = ["self", "actor", "actress"]

actors = test[test['category'].isin(actors_list)]

# actors = df_prin["category"].str.contains("act")
# directors = df_prin["category"].str.contains("dire")
# writors = df_prin["category"].str.contains("wri")
# cinemato = df_prin["category"].str.contains("cinem")

In [20]:
directors = test[test["category"].str.contains("director")]

In [21]:
actors

tconst  ordering      nconst category  \
0         tt0000001         1   nm1588970     self   
11        tt0000005         1   nm0443482    actor   
12        tt0000005         2   nm0653042    actor   
16        tt0000007         1   nm0179163    actor   
17        tt0000007         2   nm0183947    actor   
...             ...       ...         ...      ...   
58672892  tt9916856         4  nm10538647  actress   
58672897  tt9916880         1   nm2784764  actress   
58672898  tt9916880         2   nm1483166    actor   
58672899  tt9916880         3   nm0254176  actress   
58672900  tt9916880         4   nm0286175    actor   

                                          characters  
0                                             [Self]  
11                                      [Blacksmith]  
12                                       [Assistant]  
16                                         [Unknown]  
17                                         [Unknown]  
...                                              ...  
58672892                                     [Kathi]  
58672897                                     [Henry]  
58672898  [Rude Ralph, Mischievous Mike, Jolly Josh]  
58672899                            [Moody Margaret]  
58672900             [Dad, Aerobic Al, Nasty Nicola]  

[33160614 rows x 5 columns]

In [22]:
directors

tconst  ordering      nconst  category characters
1         tt0000001         2   nm0005690  director  [Unknown]
3         tt0000002         1   nm0721526  director  [Unknown]
5         tt0000003         1   nm0721526  director  [Unknown]
9         tt0000004         1   nm0721526  director  [Unknown]
13        tt0000005         3   nm0005690  director  [Unknown]
...             ...       ...         ...       ...        ...
58672874  tt9916850         5   nm1485677  director  [Unknown]
58672884  tt9916852         5   nm1485677  director  [Unknown]
58672893  tt9916856         5  nm10538645  director  [Unknown]
58672901  tt9916880         5   nm0584014  director  [Unknown]
58672902  tt9916880         6   nm0996406  director  [Unknown]

[6776469 rows x 5 columns]

In [23]:
actors.to_parquet("clean_datasets/actors.parquet")

In [24]:
directors.to_parquet("clean_datasets/directors.parquet")

In [27]:
# j'ai essayé de lancer ça ça a pris plus de 7 min j'ai give up

others = test[~test["category"].isin([actors, directors])]

In [27]:
df.dtypes

nconst            object
primaryName       object
birthYear          int64
knownForTitles    object
dtype: object

In [28]:
actors.dtypes

tconst        object
ordering       int64
nconst        object
category      object
characters    object
dtype: object

In [29]:
directors.dtypes

tconst        object
ordering       int64
nconst        object
category      object
characters    object
dtype: object

In [33]:
link = "clean_datasets/movies_cleaned.parquet"

movies = import_datasets(link, "parquet")

2023-10-26 20:58:11 INFO     Parquet loaded ! Importing movies_cleaned...


In [35]:
df_movies = movies.copy()

In [36]:
df_actors = actors.copy()

In [37]:
df_person = df.copy()

In [38]:
df_actors.head()

tconst  ordering     nconst category    characters
0   tt0000001         1  nm1588970     self        [Self]
11  tt0000005         1  nm0443482    actor  [Blacksmith]
12  tt0000005         2  nm0653042    actor   [Assistant]
16  tt0000007         1  nm0179163    actor     [Unknown]
17  tt0000007         2  nm0183947    actor     [Unknown]

In [39]:
df_person.head()

nconst      primaryName  birthYear  \
0  nm0000001     Fred Astaire       1899   
1  nm0000002    Lauren Bacall       1924   
2  nm0000003  Brigitte Bardot       1934   
3  nm0000004     John Belushi       1949   
4  nm0000005   Ingmar Bergman       1918   

                                 knownForTitles  
0  [tt0050419, tt0072308, tt0053137, tt0031983]  
1  [tt0075213, tt0037382, tt0038355, tt0117057]  
2  [tt0054452, tt0057345, tt0049189, tt0056404]  
3  [tt0080455, tt0072562, tt0078723, tt0077975]  
4  [tt0083922, tt0050976, tt0050986, tt0069467]

In [40]:
actors_names = pd.merge(
    df_actors,
    df_person,
    on = "nconst"
)

In [41]:
actors_names.head()

tconst  ordering     nconst category    characters       primaryName  \
0  tt0000001         1  nm1588970     self        [Self]        Carmencita   
1  tt0000005         1  nm0443482    actor  [Blacksmith]    Charles Kayser   
2  tt0000005         2  nm0653042    actor   [Assistant]          John Ott   
3  tt0000007         1  nm0179163    actor     [Unknown]  James J. Corbett   
4  tt0000147         1  nm0179163     self        [Self]  James J. Corbett   

   birthYear                                knownForTitles  
0       1868                        [tt0057728, tt0000001]  
1       1878                                   [tt0000005]  
2       1850                                   [tt0000005]  
3       1866  [tt0003730, tt0003116, tt0163131, tt0034778]  
4       1866  [tt0003730, tt0003116, tt0163131, tt0034778]

In [44]:
movies_actors = pd.merge(
    actors_names,
    df_movies,
    left_on = "tconst",
    right_on = "titre_id"
)

In [45]:
movies_actors.head()

tconst  ordering     nconst category    characters        primaryName  \
0  tt0093191         4  nm0092290    actor       [Homer]          Curt Bois   
1  tt0093191         1  nm0004486    actor      [Damiel]         Bruno Ganz   
2  tt0093191         3  nm0761420    actor     [Cassiel]        Otto Sander   
3  tt0093191         2  nm0231652  actress      [Marion]  Solveig Dommartin   
4  tt0052745         2  nm0187981    actor  [Jehan Daas]       Donald Crisp   

   birthYear                                knownForTitles   titre_id  \
0       1901  [tt0034583, tt0093191, tt0048162, tt0042474]  tt0093191   
1       1941  [tt0093191, tt0075675, tt2193215, tt0363163]  tt0093191   
2       1941  [tt0093191, tt0082708, tt0107209, tt0082096]  tt0093191   
3       1961  [tt0107209, tt0101458, tt0093191, tt0176836]  tt0093191   
4       1882  [tt0033729, tt0009968, tt0030287, tt0032145]  tt0052745   

           titre_str  ...  original_language          original_title  \
0    Wings of Desire  ...                 de  Der Himmel über Berlin   
1    Wings of Desire  ...                 de  Der Himmel über Berlin   
2    Wings of Desire  ...                 de  Der Himmel über Berlin   
3    Wings of Desire  ...                 de  Der Himmel über Berlin   
4  A Dog of Flanders  ...                 en       A Dog of Flanders   

  popularity  production_countries    revenue              spoken_languages  \
0      15.04              [FR, DE] 3210139.00  [de, en, es, fr, he, ja, tr]   
1      15.04              [FR, DE] 3210139.00  [de, en, es, fr, he, ja, tr]   
2      15.04              [FR, DE] 3210139.00  [de, en, es, fr, he, ja, tr]   
3      15.04              [FR, DE] 3210139.00  [de, en, es, fr, he, ja, tr]   
4       1.40                  [US]       0.00                          [en]   

     status    titleId region       cuts  
0  Released  tt0093191     FR  1980-1989  
1  Released  tt0093191     FR  1980-1989  
2  Released  tt0093191     FR  1980-1989  
3  Released  tt0093191     FR  1980-1989  
4  Released  tt0052745     FR  1960-1969  

[5 rows x 25 columns]

In [46]:
movies_actors.drop(["tconst"], inplace=True, axis=1)

In [47]:
copy = movies_actors.copy()

In [50]:
column_order = [
    "titre_id",
    "titre_str",
    "titre_date_sortie",
    "titre_duree",
    "titre_genres",
    "rating_avg",
    "rating_votes",
    "original_language",
    "original_title",
    "popularity",
    "production_countries",
    "revenue",
    "spoken_languages",
    "status",
    "titleId",
    "region",
    "cuts",
    "nconst",
    "category",
    "characters",
    "primaryName",
    "birthYear",
    "knownForTitles",
    "ordering"
]

df = copy[column_order]

df.head()

titre_id          titre_str  titre_date_sortie  titre_duree  \
0  tt0093191    Wings of Desire               1987          128   
1  tt0093191    Wings of Desire               1987          128   
2  tt0093191    Wings of Desire               1987          128   
3  tt0093191    Wings of Desire               1987          128   
4  tt0052745  A Dog of Flanders               1960           96   

                titre_genres  rating_avg  rating_votes original_language  \
0  [Drama, Fantasy, Romance]        7.90         74744                de   
1  [Drama, Fantasy, Romance]        7.90         74744                de   
2  [Drama, Fantasy, Romance]        7.90         74744                de   
3  [Drama, Fantasy, Romance]        7.90         74744                de   
4            [Drama, Family]        7.10           444                en   

           original_title  popularity  ...    titleId  region       cuts  \
0  Der Himmel über Berlin       15.04  ...  tt0093191      FR  1980-1989   
1  Der Himmel über Berlin       15.04  ...  tt0093191      FR  1980-1989   
2  Der Himmel über Berlin       15.04  ...  tt0093191      FR  1980-1989   
3  Der Himmel über Berlin       15.04  ...  tt0093191      FR  1980-1989   
4       A Dog of Flanders        1.40  ...  tt0052745      FR  1960-1969   

      nconst category    characters        primaryName birthYear  \
0  nm0092290    actor       [Homer]          Curt Bois      1901   
1  nm0004486    actor      [Damiel]         Bruno Ganz      1941   
2  nm0761420    actor     [Cassiel]        Otto Sander      1941   
3  nm0231652  actress      [Marion]  Solveig Dommartin      1961   
4  nm0187981    actor  [Jehan Daas]       Donald Crisp      1882   

                                 knownForTitles ordering  
0  [tt0034583, tt0093191, tt0048162, tt0042474]        4  
1  [tt0093191, tt0075675, tt2193215, tt0363163]        1  
2  [tt0093191, tt0082708, tt0107209, tt0082096]        3  
3  [tt0107209, tt0101458, tt0093191, tt0176836]        2  
4  [tt0033729, tt0009968, tt0030287, tt0032145]        2  

[5 rows x 24 columns]

In [62]:
df.head()

titre_str  titre_date_sortie  titre_duree  \
0    Wings of Desire               1987          128   
1    Wings of Desire               1987          128   
2    Wings of Desire               1987          128   
3    Wings of Desire               1987          128   
4  A Dog of Flanders               1960           96   

                titre_genres  rating_avg  rating_votes original_language  \
0  [Drama, Fantasy, Romance]        7.90         74744                de   
1  [Drama, Fantasy, Romance]        7.90         74744                de   
2  [Drama, Fantasy, Romance]        7.90         74744                de   
3  [Drama, Fantasy, Romance]        7.90         74744                de   
4            [Drama, Family]        7.10           444                en   

           original_title  popularity production_countries region       cuts  \
0  Der Himmel über Berlin       15.04             [FR, DE]     FR  1980-1989   
1  Der Himmel über Berlin       15.04             [FR, DE]     FR  1980-1989   
2  Der Himmel über Berlin       15.04             [FR, DE]     FR  1980-1989   
3  Der Himmel über Berlin       15.04             [FR, DE]     FR  1980-1989   
4       A Dog of Flanders        1.40                 [US]     FR  1960-1969   

     characters        primaryName  birthYear  \
0       [Homer]          Curt Bois       1901   
1      [Damiel]         Bruno Ganz       1941   
2     [Cassiel]        Otto Sander       1941   
3      [Marion]  Solveig Dommartin       1961   
4  [Jehan Daas]       Donald Crisp       1882   

                                 knownForTitles  ordering  
0  [tt0034583, tt0093191, tt0048162, tt0042474]         4  
1  [tt0093191, tt0075675, tt2193215, tt0363163]         1  
2  [tt0093191, tt0082708, tt0107209, tt0082096]         3  
3  [tt0107209, tt0101458, tt0093191, tt0176836]         2  
4  [tt0033729, tt0009968, tt0030287, tt0032145]         2

In [ ]:
df[df["primaryName"].str.contains("rard Depardieu")]

In [66]:
df["primaryName"].value_counts()

primaryName
GÃ©rard Depardieu        122
Catherine Deneuve         99
Michel Piccoli            92
Isabelle Huppert          91
Michael Caine             87
                        ... 
NÃºria Prims               1
Urbanus                    1
Sylvia Millecam            1
Babette van Veen           1
Elizabett ConceiÃ§Ã£o      1
Name: count, Length: 62368, dtype: int64

**JME SUIS ARRETÉ LA**


In [30]:
# link = "big_dataframe/actors.parquet"
# df_og = import_datasets(
#     link,
#     types="parquet",
# )
# cleaning = DataCleaner()
# df = cleaning.fix_values(df_og, "fix_n")
# # df = cleaning.fix_values(df1, "fix_encode")

2023-10-26 16:51:09 INFO     Parquet loaded ! Importing actors...
2023-10-26 16:51:10 INFO     Fixing N values...


In [32]:
df["person_birthdate"] = df["person_birthdate"].astype("int64")

In [33]:
df.isna().sum()

person_id           0
person_name         0
person_birthdate    0
person_role         0
person_index        0
person_film         0
dtype: int64

In [34]:
df.dtypes

person_id           object
person_name         object
person_birthdate     int64
person_role         object
person_index         int64
person_film         object
dtype: object

In [35]:
df["person_film"] = df["person_film"].str.split(",")

In [36]:
df["person_role"] = np.where(df["person_role"] == 0, "Unknown", df["person_role"])

In [37]:
df.isna().sum()

person_id             0
person_name           0
person_birthdate      0
person_role           0
person_index          0
person_film         739
dtype: int64

In [38]:
df["person_film"] = np.where(df["person_film"].isna(), "Unknown", df["person_film"])

In [39]:
mf = df.copy()

In [40]:
mf["person_role"] = mf["person_role"].apply(decode_clean_actors).str.split(",")

In [41]:
mf.head()

person_id   person_name  person_birthdate      person_role  person_index  \
0  nm0000001  Fred Astaire              1899     [Guy Holden]             1   
1  nm0000001  Fred Astaire              1899    [Huck Haines]             2   
2  nm0000001  Fred Astaire              1899  [Jerry Travers]             1   
3  nm0000001  Fred Astaire              1899     [Bake Baker]             1   
4  nm0000001  Fred Astaire              1899  [Lucky Garnett]             1   

                                    person_film  
0  [tt0050419, tt0072308, tt0053137, tt0031983]  
1  [tt0050419, tt0072308, tt0053137, tt0031983]  
2  [tt0050419, tt0072308, tt0053137, tt0031983]  
3  [tt0050419, tt0072308, tt0053137, tt0031983]  
4  [tt0050419, tt0072308, tt0053137, tt0031983]

In [ ]:
mf[mf["person_name"].str.contains('Eddie Murphy')]

In [43]:
mf.shape

(1183377, 6)

In [44]:
# merge avec movies

In [45]:
link = "clean_datasets/movies_cleaned.parquet"
link_principals = "movies_datasets/title_principals.tsv"

movies_cleaned = import_datasets(link, "parquet")
principals = import_datasets(link_principals, "pandas", sep="\t")

2023-10-26 16:51:20 INFO     Parquet loaded ! Importing movies_cleaned...
2023-10-26 16:51:20 INFO     Pandas loaded ! Importing title_principals...


In [46]:
principals = principals[["tconst", "nconst"]]

principals.head()

tconst     nconst
0  tt0000001  nm1588970
1  tt0000001  nm0005690
2  tt0000001  nm0374658
3  tt0000002  nm0721526
4  tt0000002  nm1335271

In [122]:
movies_with_personid = pd.merge(
    movies_cleaned,
    principals,
    left_on = "titre_id",
    right_on = "tconst",
)
#  ICi on récuperer tous les acteurs avec leurs titres

In [123]:
condi2 = (
    (movies_with_personid["titre_id"].str.contains("tt0068991")) &
    (movies_with_personid["nconst"].str.contains("nm0000367"))
)
movies_with_personid[condi2]

titre_id         titre_str  titre_date_sortie  titre_duree  \
58260  tt0068991  Nathalie Granger               1972           83   

      titre_genres  rating_avg  rating_votes original_language  \
58260      [Drama]        6.70           654                fr   

         original_title  popularity production_countries  revenue  \
58260  Nathalie Granger        1.83                 [FR]     0.00   

      spoken_languages    status    titleId region       cuts     tconst  \
58260             [fr]  Released  tt0068991     FR  1970-1979  tt0068991   

          nconst  
58260  nm0000367

In [124]:
movies_with_personid["titre_id"].unique().size

36228

In [134]:
movies_with_personid

titre_id  titre_str  titre_date_sortie  titre_duree  \
0       tt0062989  Fort Utah               1967           84   
1       tt0062989  Fort Utah               1967           84   
2       tt0062989  Fort Utah               1967           84   
3       tt0062989  Fort Utah               1967           84   
4       tt0062989  Fort Utah               1967           84   
...           ...        ...                ...          ...   
352799  tt9916362      Coven               2020           92   
352800  tt9916362      Coven               2020           92   
352801  tt9916362      Coven               2020           92   
352802  tt9916362      Coven               2020           92   
352803  tt9916362      Coven               2020           92   

            titre_genres  rating_avg  rating_votes original_language  \
0              [Western]        5.00           303                en   
1              [Western]        5.00           303                en   
2              [Western]        5.00           303                en   
3              [Western]        5.00           303                en   
4              [Western]        5.00           303                en   
...                  ...         ...           ...               ...   
352799  [Drama, History]        6.40          5571                eu   
352800  [Drama, History]        6.40          5571                eu   
352801  [Drama, History]        6.40          5571                eu   
352802  [Drama, History]        6.40          5571                eu   
352803  [Drama, History]        6.40          5571                eu   

       original_title  popularity production_countries  revenue  \
0           Fort Utah        3.05                 [US]     0.00   
1           Fort Utah        3.05                 [US]     0.00   
2           Fort Utah        3.05                 [US]     0.00   
3           Fort Utah        3.05                 [US]     0.00   
4           Fort Utah        3.05                 [US]     0.00   
...               ...         ...                  ...      ...   
352799       Akelarre        8.63         [AR, ES, FR]     0.00   
352800       Akelarre        8.63         [AR, ES, FR]     0.00   
352801       Akelarre        8.63         [AR, ES, FR]     0.00   
352802       Akelarre        8.63         [AR, ES, FR]     0.00   
352803       Akelarre        8.63         [AR, ES, FR]     0.00   

       spoken_languages    status    titleId region       cuts     tconst  \
0              [de, en]  Released  tt0062989     FR  1960-1969  tt0062989   
1              [de, en]  Released  tt0062989     FR  1960-1969  tt0062989   
2              [de, en]  Released  tt0062989     FR  1960-1969  tt0062989   
3              [de, en]  Released  tt0062989     FR  1960-1969  tt0062989   
4              [de, en]  Released  tt0062989     FR  1960-1969  tt0062989   
...                 ...       ...        ...    ...        ...        ...   
352799         [eu, es]  Released  tt9916362     FR      >2020  tt9916362   
352800         [eu, es]  Released  tt9916362     FR      >2020  tt9916362   
352801         [eu, es]  Released  tt9916362     FR      >2020  tt9916362   
352802         [eu, es]  Released  tt9916362     FR      >2020  tt9916362   
352803         [eu, es]  Released  tt9916362     FR      >2020  tt9916362   

           nconst  
0       nm0941700  
1       nm0409869  
2       nm0562920  
3       nm0103722  
4       nm0751245  
...           ...  
352799  nm1893148  
352800  nm3471432  
352801  nm2970042  
352802  nm4065853  
352803  nm1086949  

[352804 rows x 19 columns]

In [153]:
actors_merged = pd.merge(
    mf,
    movies_with_personid,
    left_on = "person_id",
    right_on = "nconst",
    how="left"
)

gerard = movies_with_personid[movies_with_personid.nconst.isin(mf.person_id)]
# gerard.head()
# actors_merged = actors_merged.drop("nconst", axis=1)

In [157]:
gerard

titre_id                         titre_str  titre_date_sortie  \
1        tt0062989                         Fort Utah               1967   
2        tt0062989                         Fort Utah               1967   
3        tt0062989                         Fort Utah               1967   
4        tt0062989                         Fort Utah               1967   
8        tt0062989                         Fort Utah               1967   
...            ...                               ...                ...   
352789  tt15398694  Convergence: Courage in a Crisis               2021   
352795   tt9916362                             Coven               2020   
352796   tt9916362                             Coven               2020   
352797   tt9916362                             Coven               2020   
352798   tt9916362                             Coven               2020   

        titre_duree      titre_genres  rating_avg  rating_votes  \
1                84         [Western]        5.00           303   
2                84         [Western]        5.00           303   
3                84         [Western]        5.00           303   
4                84         [Western]        5.00           303   
8                84         [Western]        5.00           303   
...             ...               ...         ...           ...   
352789          113     [Documentary]        6.00           300   
352795           92  [Drama, History]        6.40          5571   
352796           92  [Drama, History]        6.40          5571   
352797           92  [Drama, History]        6.40          5571   
352798           92  [Drama, History]        6.40          5571   

       original_language                    original_title  popularity  \
1                     en                         Fort Utah        3.05   
2                     en                         Fort Utah        3.05   
3                     en                         Fort Utah        3.05   
4                     en                         Fort Utah        3.05   
8                     en                         Fort Utah        3.05   
...                  ...                               ...         ...   
352789                en  Convergence: Courage in a Crisis        4.67   
352795                eu                          Akelarre        8.63   
352796                eu                          Akelarre        8.63   
352797                eu                          Akelarre        8.63   
352798                eu                          Akelarre        8.63   

       production_countries  revenue              spoken_languages    status  \
1                      [US]     0.00                      [de, en]  Released   
2                      [US]     0.00                      [de, en]  Released   
3                      [US]     0.00                      [de, en]  Released   
4                      [US]     0.00                      [de, en]  Released   
8                      [US]     0.00                      [de, en]  Released   
...                     ...      ...                           ...       ...   
352789             [GB, US]     0.00  [en, hi, zh, es, pt, fa, fr]  Released   
352795         [AR, ES, FR]     0.00                      [eu, es]  Released   
352796         [AR, ES, FR]     0.00                      [eu, es]  Released   
352797         [AR, ES, FR]     0.00                      [eu, es]  Released   
352798         [AR, ES, FR]     0.00                      [eu, es]  Released   

           titleId region       cuts      tconst      nconst  
1        tt0062989     FR  1960-1969   tt0062989   nm0409869  
2        tt0062989     FR  1960-1969   tt0062989   nm0562920  
3        tt0062989     FR  1960-1969   tt0062989   nm0103722  
4        tt0062989     FR  1960-1969   tt0062989   nm0751245  
8        tt0062989     FR  1960-1969   tt0062989   nm0528121  
...            ...    ...        ...         ...         ...  
352789  tt153

In [158]:
actors_merged = pd.merge(
    gerard,
    mf,
    left_on = "nconst",
    right_on = "person_id",
)

In [159]:
actors_merged

titre_id  titre_str  titre_date_sortie  titre_duree  \
0        tt0062989  Fort Utah               1967           84   
1        tt0062989  Fort Utah               1967           84   
2        tt0062989  Fort Utah               1967           84   
3        tt0062989  Fort Utah               1967           84   
4        tt0062989  Fort Utah               1967           84   
...            ...        ...                ...          ...   
3116184  tt9916362      Coven               2020           92   
3116185  tt9916362      Coven               2020           92   
3116186  tt9916362      Coven               2020           92   
3116187  tt9916362      Coven               2020           92   
3116188  tt9916362      Coven               2020           92   

             titre_genres  rating_avg  rating_votes original_language  \
0               [Western]        5.00           303                en   
1               [Western]        5.00           303                en   
2               [Western]        5.00           303                en   
3               [Western]        5.00           303                en   
4               [Western]        5.00           303                en   
...                   ...         ...           ...               ...   
3116184  [Drama, History]        6.40          5571                eu   
3116185  [Drama, History]        6.40          5571                eu   
3116186  [Drama, History]        6.40          5571                eu   
3116187  [Drama, History]        6.40          5571                eu   
3116188  [Drama, History]        6.40          5571                eu   

        original_title  popularity  ... region       cuts     tconst  \
0            Fort Utah        3.05  ...     FR  1960-1969  tt0062989   
1            Fort Utah        3.05  ...     FR  1960-1969  tt0062989   
2            Fort Utah        3.05  ...     FR  1960-1969  tt0062989   
3            Fort Utah        3.05  ...     FR  1960-1969  tt0062989   
4            Fort Utah        3.05  ...     FR  1960-1969  tt0062989   
...                ...         ...  ...    ...        ...        ...   
3116184       Akelarre        8.63  ...     FR      >2020  tt9916362   
3116185       Akelarre        8.63  ...     FR      >2020  tt9916362   
3116186       Akelarre        8.63  ...     FR      >2020  tt9916362   
3116187       Akelarre        8.63  ...     FR      >2020  tt9916362   
3116188       Akelarre        8.63  ...     FR      >2020  tt9916362   

             nconst   person_id       person_name person_birthdate  \
0         nm0409869   nm0409869      John Ireland             1914   
1         nm0409869   nm0409869      John Ireland             1914   
2         nm0409869   nm0409869      John Ireland             1914   
3         nm0409869   nm0409869      John Ireland             1914   
4         nm0409869   nm0409869      John Ireland             1914   
...             ...         ...               ...              ...   
3116184   nm3766704   nm3766704  Amaia Aberasturi             1997   
3116185   nm3766704   nm3766704  Amaia Aberasturi             1997   
3116186   nm3766704   nm3766704  Amaia Aberasturi             1997   
3116187   nm3766704   nm3766704  Amaia Aberasturi             1997   
3116188  nm10678594  nm10678594     Garazi Urkola                0   

                 person_role person_index  \
0        [Pfc. Windy Craven]            4   
1              [Duke Martin]            1   
2              [Paul Lester]            1   
3                  [Fantail]            4   
4              [Jack Burden]            2   
...                      ...          ...   
3116184              [Leire]            1   
3116185            [Unknown]            1   
3116186             [Begoña]            1   
3116187                [Ana]            1   
3116188            [Katalin]            4   

                                           person_film  
0         [tt0040724, tt0054331, tt0039748, tt00

In [145]:
actors_merged["titre_id"].unique().size

35960

In [160]:
condi2 = (
    (actors_merged["titre_id"].str.contains("tt0068991")) &
    (actors_merged["nconst"].str.contains("nm0000367"))
)
actors_merged[condi2]

titre_id         titre_str  titre_date_sortie  titre_duree  \
1156006  tt0068991  Nathalie Granger               1972           83   
1156007  tt0068991  Nathalie Granger               1972           83   
1156008  tt0068991  Nathalie Granger               1972           83   
1156009  tt0068991  Nathalie Granger               1972           83   
1156010  tt0068991  Nathalie Granger               1972           83   
...            ...               ...                ...          ...   
1156152  tt0068991  Nathalie Granger               1972           83   
1156153  tt0068991  Nathalie Granger               1972           83   
1156154  tt0068991  Nathalie Granger               1972           83   
1156155  tt0068991  Nathalie Granger               1972           83   
1156156  tt0068991  Nathalie Granger               1972           83   

        titre_genres  rating_avg  rating_votes original_language  \
1156006      [Drama]        6.70           654                fr   
1156007      [Drama]        6.70           654                fr   
1156008      [Drama]        6.70           654                fr   
1156009      [Drama]        6.70           654                fr   
1156010      [Drama]        6.70           654                fr   
...              ...         ...           ...               ...   
1156152      [Drama]        6.70           654                fr   
1156153      [Drama]        6.70           654                fr   
1156154      [Drama]        6.70           654                fr   
1156155      [Drama]        6.70           654                fr   
1156156      [Drama]        6.70           654                fr   

           original_title  popularity  ... region       cuts     tconst  \
1156006  Nathalie Granger        1.83  ...     FR  1970-1979  tt0068991   
1156007  Nathalie Granger        1.83  ...     FR  1970-1979  tt0068991   
1156008  Nathalie Granger        1.83  ...     FR  1970-1979  tt0068991   
1156009  Nathalie Granger        1.83  ...     FR  1970-1979  tt0068991   
1156010  Nathalie Granger        1.83  ...     FR  1970-1979  tt0068991   
...                   ...         ...  ...    ...        ...        ...   
1156152  Nathalie Granger        1.83  ...     FR  1970-1979  tt0068991   
1156153  Nathalie Granger        1.83  ...     FR  1970-1979  tt0068991   
1156154  Nathalie Granger        1.83  ...     FR  1970-1979  tt0068991   
1156155  Nathalie Granger        1.83  ...     FR  1970-1979  tt0068991   
1156156  Nathalie Granger        1.83  ...     FR  1970-1979  tt0068991   

            nconst  person_id       person_name person_birthdate  \
1156006  nm0000367  nm0000367  Gérard Depardieu             1948   
1156007  nm0000367  nm0000367  Gérard Depardieu             1948   
1156008  nm0000367  nm0000367  Gérard Depardieu             1948   
1156009  nm0000367  nm0000367  Gérard Depardieu             1948   
1156010  nm0000367  nm0000367  Gérard Depardieu             1948   
...            ...        ...               ...              ...   
1156152  nm0000367  nm0000367  Gérard Depardieu             1948   
1156153  nm0000367  nm0000367  Gérard Depardieu             1948   
1156154  nm0000367  nm0000367  Gérard Depardieu             1948   
1156155  nm0000367  nm0000367  Gérard Depardieu             1948   
1156156  nm0000367  nm0000367  Gérard Depardieu             1948   

                  person_role person_index  \
1156006            [Salesman]            3   
1156007  [Lhomme de la plage]            3   
1156008      [Pierre Vaucher]            2   
1156009         [Jean-Claude]            1   
1156010       [Jean Lavallee]            4   
...                       ...          ...   
1156152           [Saintonge]            4   
1156153      [Gabriel Carvin]            1   
1156154             [Patrick]            3   
1156155        [Raoul Taupin]            1   
1156156    [Gérard Depardieu]            2   

                                          person_film  
1

In [82]:
df = actors_merged.copy()

In [83]:
df.head()

titre_id  titre_str  titre_date_sortie  titre_duree titre_genres  \
0  tt0062989  Fort Utah               1967           84    [Western]   
1  tt0062989  Fort Utah               1967           84    [Western]   
2  tt0062989  Fort Utah               1967           84    [Western]   
3  tt0062989  Fort Utah               1967           84    [Western]   
4  tt0062989  Fort Utah               1967           84    [Western]   

   rating_avg  rating_votes original_language original_title  popularity  ...  \
0        5.00           303                en      Fort Utah        3.05  ...   
1        5.00           303                en      Fort Utah        3.05  ...   
2        5.00           303                en      Fort Utah        3.05  ...   
3        5.00           303                en      Fort Utah        3.05  ...   
4        5.00           303                en      Fort Utah        3.05  ...   

     status    titleId region       cuts  person_id   person_name  \
0  Released  tt0062989     FR  1960-1969  nm0409869  John Ireland   
1  Released  tt0062989     FR  1960-1969  nm0409869  John Ireland   
2  Released  tt0062989     FR  1960-1969  nm0409869  John Ireland   
3  Released  tt0062989     FR  1960-1969  nm0409869  John Ireland   
4  Released  tt0062989     FR  1960-1969  nm0409869  John Ireland   

  person_birthdate          person_role person_index  \
0             1914  [Pfc. Windy Craven]            4   
1             1914        [Duke Martin]            1   
2             1914        [Paul Lester]            1   
3             1914            [Fantail]            4   
4             1914        [Jack Burden]            2   

                                    person_film  
0  [tt0040724, tt0054331, tt0039748, tt0041113]  
1  [tt0040724, tt0054331, tt0039748, tt0041113]  
2  [tt0040724, tt0054331, tt0039748, tt0041113]  
3  [tt0040724, tt0054331, tt0039748, tt0041113]  
4  [tt0040724, tt0054331, tt0039748, tt0041113]  

[5 rows x 23 columns]

In [ ]:
condi2 = (
    (principals["tconst"].str.contains("tt0068991")) &
    (principals["nconst"].str.contains("nm0000367"))
)


In [84]:
len(df[df["person_name"].str.contains("Gérard Depardieu")])

18724

In [72]:
df.isna().sum()

titre_id                0
titre_str               0
titre_date_sortie       0
titre_duree             0
titre_genres            0
rating_avg              0
rating_votes            0
original_language       0
original_title          0
popularity              0
production_countries    0
revenue                 0
spoken_languages        0
status                  0
titleId                 0
region                  0
cuts                    0
person_id               0
person_name             0
person_birthdate        0
person_role             0
person_index            0
person_film             0
dtype: int64